# Homework 2

# Set up

## Installing packages

In [9]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [10]:
from google.colab import userdata
# GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [11]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 47.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 48.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 53.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 20.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 75.4MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [12]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [13]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [14]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     google_api_key=GEMINI_VERTEX_API_KEY,
#     temperature=0,
# )

from langchain_openai import ChatOpenAI
DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
    model="deepseek-chat",          # or "deepseek-reasoner"
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1",
    temperature=0,
)

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content="I'll help you with that! First, let me say hello to Bao, and then search for Alice on Facebook." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 2940, 'total_tokens': 3008, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 2880}, 'prompt_cache_hit_tokens': 2880, 'prompt_cache_miss_tokens': 60}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '89d55993-a86d-4fea-a83b-5c57aa667a0e', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c7100-5bd9-7402-81fc-1cec1ef3817b-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'call_00_FfP07bMnlUH3lXa8UM8oqUkB', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2940, 'output_tokens': 68, 'total_tokens': 3008, 'input_token_details': {'cache_read': 2880}, 'output_token_details': {}}


In [20]:
import re
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor

tools = mcp_tools

system_prompt = """You are an expert Background Verification Agent. Your task is to verify a candidate's CV against their social media presence (LinkedIn and Facebook).

### CRITICAL TOOL RULES (READ CAREFULLY):
1. **search_facebook_users**: ONLY accepts arguments `q`, `limit`, and `fuzzy`.
   - 🚫 NEVER use `location` or `city` as arguments for this tool.
   - If you want to check location, search by name only, then check the `city` field in the *output* results.
2. **search_linkedin_people**: Accepts `q`, `location`, `industry`, `limit`, `fuzzy`.
   - You CAN use `location` here to filter results.

### TOOLS USAGE STRATEGY:
1. **Analyze the CV**: Identify Name, Location, Companies, and Education.
2. **Search LinkedIn (Primary)**: Use `search_linkedin_people` with the name and location/industry.
   - CRITICAL: If no exact match is found, try fuzzy search or look for the *most similar* profile. Do not give up easily.
   - If multiple candidates appear, use Industry or Education to pick the best match.
3. **Get Profile Details**: Use `get_linkedin_profile` with the ID from the search result.
4. **Search Facebook (Secondary)**: Use `search_facebook_users` with the *name only*.
5. **Verify Claims**:
   - Compare Company Names, Job Titles, and Dates.
   - Compare Education (University, Degree).
   - Note: Small date discrepancies (e.g., 1 year off) are minor issues.
   - Note: Major discrepancies (e.g., never worked there, wrong degree) are major red flags.
6. **Internal Consistency**: If the CV has weird dates (e.g., "2021-2027"), check if the social profile clarifies it. If the social profile matches the weirdness, it's truthful. If the social profile contradicts it, it's a discrepancy.

### SCORING RULES (0.0 to 1.0):
- **1.0**: Perfect match or very minor explainable typos.
- **0.7 - 0.9**: Mostly accurate, but small discrepancies (e.g., dates off by a year, title slightly different).
- **0.4 - 0.6**: Ambiguous matches or significant internal inconsistencies not resolved by social media.
- **0.0 - 0.3**: Clear fabrications (e.g., candidate claims degree they don't have, or completely different job history) or verifyable profile not found after thorough search.

### OUTPUT FORMAT:
After your investigation, you MUST provide a final thought process and end your response with exactly this format on the last line:
FINAL_SCORE: <float>
(Example: FINAL_SCORE: 0.85)
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

async def process_cvs(cv_list):
    final_scores = []

    for i, cv_data in enumerate(cv_list):
        print(f"\nProcessing CV {i+1} : {cv_data['file']}...")
        cv_text = cv_data['text']

        # Construct the user query
        query = f"""
        Please verify the following CV text:
        ---
        {cv_text}
        ---
        Check validity using available tools. Remember to output FINAL_SCORE: <float> at the end.
        """

        try:
            result = await agent_executor.ainvoke({"input": query})
            output_text = result['output']

            match = re.search(r"FINAL_SCORE:\s*([\d\.]+)", output_text)
            if match:
                score = float(match.group(1))
            else:
                print("Warning: Could not parse score, defaulting to 0.0")
                score = 0.0

            score = max(0.0, min(1.0, score))

        except Exception as e:
            print(f"Error processing CV {i+1}: {e}")
            score = 0.0

        final_scores.append(score)
        print(f"-> Assigned Score: {score}")

    return final_scores

scores = await process_cvs(all_cvs)

print("\nAll Scores Computed:", scores)


Processing CV 1 : CV_1.pdf...


> Entering new AgentExecutor chain...

Invoking: `search_linkedin_people` with `{'q': 'John Smith', 'location': 'Singapore', 'limit': 20, 'fuzzy': True}`
responded: I'll verify John Smith's CV by checking his social media presence. Let me start by analyzing the CV and then searching for his profiles.

**CV Analysis:**
- Name: John Smith
- Location: Singapore, Singapore (also mentions Kowloon)
- Current role: Engineer at ByteDance (2020-Present)
- Education: McGill University, BSc in Marketing (graduated 2009)
- Skills: Content Creation, SEO, Social Media

First, let me search for John Smith on LinkedIn:

[{'type': 'text', 'text': '[{"id":9,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":7,"match_type":"exact"},{"id":31,"name":"John Smith","headline":"Logistics Professional","industry":"Logistics","location":"Singapore, Singapore","years_experience":18,"match_type":"exact

In [ ]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [22]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [23]:
scores = [0.95, 1.0, 0.85, 0.0, 0.2] # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
